## Notebook to evaluate consistency and stability measure of the mode

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
import os
from os.path import join as pj

import numpy as np
import pyrootutils

pyrootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)

from src.shared_utils.utils_experiments import  load_model_dataset
from torch.utils.data import DataLoader
from src.learning.data.FunnybirdsDataset import FunnyBirdsDataset
from src.evaluation.interpretability_metrics import attribute_analysis, compute_consistency

In [ ]:
data_path = "/workspaces/intrainpretable/data/FunnyBirds"
path_sim = #add path to the folder containing model trained on the FunnyBrid dataset
model, _ = load_model_dataset(path_sim=path_sim,set="test")

In [ ]:
transforms = None
test_dataset = FunnyBirdsDataset(
        data_path,
        "test",
        get_part_map=True,
        transform=transforms,
        eval_funny_birds=True,
    )
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
list_all_part, list_all_presence, target_all = attribute_analysis(model,test_dataset, test_loader, add_noise = False)
all_proto_part_mask= np.stack(list_all_part)
all_part_presence = np.stack(list_all_presence)

In [ ]:
all_proto_part_mask.shape

In [ ]:
all_part_presence.shape

In [ ]:
means_score = compute_consistency(all_proto_part_mask, all_part_presence, target_all, threshold = 0.8)
print("mean consistency score",means_score)

In [ ]:
list_all_part_noise, list_all_presence_noise, target_all_noise = attribute_analysis(model,test_dataset, test_loader, add_noise = True)
all_proto_part_mask_noise= np.stack(list_all_part_noise)
all_part_presence_noise = np.stack(list_all_presence_noise)

In [ ]:
# idx_proto_noise = all_proto_part_mask_noise
proto_noise = np.argwhere(all_proto_part_mask_noise.sum(axis=(0,2))>0).squeeze()
proto_noise = proto_noise.tolist()
print("proto_noise",len(proto_noise))
proto_normal = np.argwhere(all_proto_part_mask.sum(axis=(0,2))>0).squeeze()
proto_normal = proto_normal.tolist()
print("proto_normal",len(proto_normal))
union_proto = list(set(proto_noise).union(set(proto_normal)))


In [ ]:
stability_score = (all_proto_part_mask_noise[:,union_proto,:] ==all_proto_part_mask[:,union_proto,:]).sum() / np.size(all_proto_part_mask_noise[:,union_proto,:])

In [ ]:
print("mean stability score",stability_score)